In [1]:
import pandas as pd
import numpy as np
import glob
import librosa
import torchaudio.transforms
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import torch
import torchaudio
import os

In [2]:
DATASET_PATH = '../data/raw/esc50'

In [3]:
# Read the metadata
meta = pd.read_csv(f'{DATASET_PATH}/meta/esc50.csv')
meta.head()

,filename,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,1,0,dog,True,100032,A
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A


In [4]:
meta = meta[meta['category'] != 'chirping_birds']

In [5]:
# Read files with librosa
def read_audio(file):
    audio, sr = librosa.load(file, sr=None)
    return audio, sr
# Resample the adio to 32000 Hz using librosa
def resample_audio(audio, sr, new_sr):
    resampled_audio = librosa.resample(audio, orig_sr = sr, target_sr=new_sr)
    return resampled_audio


In [6]:
import soundfile as sf
new_dir = f'{DATASET_PATH}/train_audio'

# filename, labels, rating, primary_labels
os.makedirs(new_dir, exist_ok=True)

for filename in tqdm(meta['filename']):
    audio, sr = read_audio(f'{DATASET_PATH}/audio/{filename}')
    resampled_audio = resample_audio(audio, sr, 32000)

    os.makedirs(f'{new_dir}', exist_ok=True)
    sf.write(f'{new_dir}/{filename}', resampled_audio, 32000)


  0%|          | 0/1960 [00:00<?, ?it/s]

In [7]:
# write the metadata
new_metadata = pd.DataFrame({
    'filename': meta['filename'],
    "primary_label": ['silent'] * len(meta),
    "secondary_labels": ['[]'] * len(meta),
    # "labels": ['[]'] * len(meta),  # Provide a list of empty strings with the same length as meta
    "rating": [5] * len(meta) 
})
new_metadata.head()

,filename,primary_label,secondary_labels,rating
0,1-100032-A-0.wav,silent,[],5
2,1-100210-A-36.wav,silent,[],5
3,1-100210-B-36.wav,silent,[],5
4,1-101296-A-19.wav,silent,[],5
5,1-101296-B-19.wav,silent,[],5


In [8]:
# write metadata to csv
new_metadata.to_csv(f'{DATASET_PATH}/train_metadata.csv', index=False)